In [43]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import csv

# Constants
WEEKSEC = 604800
LIGHTSPEED = 299792458

# Helper functions
def gps_time_from_nanos(t):
    gpsepoch = datetime(1980, 1, 6, 0, 0, 0, tzinfo=datetime.timezone.utc)
    return (datetime.utcfromtimestamp(t * 1e-9) - gpsepoch) // timedelta(seconds=1)

def read_and_process_file(filepath):
    measurements = []
    if filepath.endswith('.txt'):
        # Plain text parsing logic
        with open(filepath, 'r') as file:
            for line in file:
                parts = line.strip().split(',')
                # Assume certain indexes; adjust as necessary
                if 'G' in parts[0]:  # Checking for GPS-related data
                    measurements.append(parse_line(parts))
    elif filepath.endswith('.nmea'):
        # NMEA parsing logic
        with open(filepath, 'r') as file:
            for line in file:
                if line.startswith('$GPGSV'):
                    parts = line.strip().split(',')
                    # Parse NMEA GPGSV for satellite data
                    measurements.extend(parse_nmea_gpgsv(parts))
    elif filepath.endswith('.24o'):
        # RINEX parsing logic
        with open(filepath, 'r') as file:
            headers = read_rinex_headers(file)
            measurements.extend(read_rinex_obs_data(file, headers))
    return measurements

def parse_line(parts):
    # Check if parts has enough elements
    if len(parts) >= 8:
        return {
            'GPS Time': parts[0],  # Modify according to your data structure
            'SatPRN (ID)': parts[1],
            'Sat.X': parts[2],
            'Sat.Y': parts[3],
            'Sat.Z': parts[4],
            'Pseudo-Range': parts[5],
            'CN0': parts[6],
            'Doppler': parts[7]
        }
    else:
        # Handle case where parts doesn't have enough elements
        print("Error: Insufficient elements in parts:", parts)
        return None

def read_rinex_headers(file):
    headers = []
    for line in file:
        if line.startswith('#'):
            headers.append(line.strip())
        else:
            break  # Exit loop when non-header line is encountered
    return headers

def read_rinex_obs_data(file, headers):
    observations = []
    for line in file:
        # Example logic to read observation data, adjust as needed
        if not line.startswith('#'):
            obs_data = line.strip().split()
            observations.append(obs_data)
    return observations


# Example usage
filepaths = ['C:/Users/דניאל ריבני/Desktop/רובוטים/Walking/gnss_log_2024_04_13_19_52_00.txt'
, 'C:/Users/דניאל ריבני/Desktop/רובוטים/Walking/gnss_log_2024_04_13_19_52_00.nmea', 'C:/Users/דניאל ריבני/Desktop/רובוטים/Walking/gnss_log_2024_04_13_19_52_00.24o']  # Update with actual file paths
all_measurements = []
for filepath in filepaths:
    all_measurements.extend(read_and_process_file(filepath))

measurements_df = pd.DataFrame(all_measurements)
measurements_df.to_csv('output_gps_data.csv', index=False)

print("Data processing complete and saved to 'output_gps_data.csv'.")


Error: Insufficient elements in parts: ['# Gyro', 'utcTimeMillis', 'elapsedRealtimeNanos', 'GyroXRadPerSec', 'GyroYRadPerSec', 'GyroZRadPerSec']


AttributeError: 'NoneType' object has no attribute 'keys'